In [1]:
"""
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/Shareddrives/LIBS/LIBS_library
"""

import numpy as np

# data

In [2]:
from src.visualization import IndexType

INDEX = IndexType.HORIZONTAL_SNAKE
DIM = (266, 500)

In [3]:
"""from src.preprocessing import match_wavelengths

s1_wavelengths = np.load(open('data/s1_wavelengths.npy', 'rb'))
s2_wavelengths = np.load(open('data/s2_wavelengths.npy', 'rb'))
s1_spectra = np.load(open('data/s1.npy', 'rb'))
s2_spectra = np.load(open('data/s2.npy', 'rb'))

y_true, y_pred = s1_spectra, s1_spectra
input = s2_spectra
calibration = s1_wavelengths
del s1_spectra, s2_spectra, s1_wavelengths, s2_wavelengths"""

"from src.preprocessing import match_wavelengths\n\ns1_wavelengths = np.load(open('data/s1_wavelengths.npy', 'rb'))\ns2_wavelengths = np.load(open('data/s2_wavelengths.npy', 'rb'))\ns1_spectra = np.load(open('data/s1.npy', 'rb'))\ns2_spectra = np.load(open('data/s2.npy', 'rb'))\n\ny_true, y_pred = s1_spectra, s1_spectra\ninput = s2_spectra\ncalibration = s1_wavelengths\ndel s1_spectra, s2_spectra, s1_wavelengths, s2_wavelengths"

# layout

In [4]:
import dash
import dash_bootstrap_components as dbc
from dash import html
from dash import dcc
from jupyter_dash import JupyterDash
from src.rowwise_metrics import rowwise_cosine, rowwise_euclid, rowwise_kl_divergence
from src.custom_components import place_in_container, make_dropdown, make_plot, make_table, make_tooltip_title, make_toggle
from src.visualization import plot_spectra

METRICS = {
    'cosine distance'            : rowwise_cosine,
    'euclidean distance'         : rowwise_euclid,
    'kullback-leibler divergence': rowwise_kl_divergence,
}

# TODO what if calibration

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SLATE, dbc_css])
app.title = 'LIBS Error Analysis'
app.layout = \
place_in_container([

    # intensity map & error map 
    # TODO maybe add table
    dbc.Row([
        dbc.Col([
            make_plot(
                dbc.Row([
                    dbc.Col([
                        make_tooltip_title('Original map', 'y_map_title', 'TBD'),
                    ]),
                    dbc.Col([
                        make_toggle('', 'Suppress outliers', 'y_map_outlier_toggle'),
                    ]),
                    dbc.Col([
                        make_toggle('y_true', 'y_pred', 'y_map_toggle'),
                    ]),
                ]),
                'y_map',
            ),
        ], width=6),
        dbc.Col([
            make_plot(
                dbc.Row([
                    dbc.Col([
                        make_tooltip_title('Error map', 'error_map_title', 'TBD'),
                    ]),
                    dbc.Col([
                        make_dropdown('error_metric_dropdown', list(METRICS.keys())),
                    ]),
                    dbc.Col([
                        make_toggle('', 'Suppress outliers', 'error_map_outlier_toggle'),
                    ]),
                ]),
                'error_map',
            ),
        ], width=6),
    ], align='center'),
    html.Br(),

    # input and output
    dbc.Row([
        dbc.Col([
            make_plot(
                make_tooltip_title('Input spectrum', 'X_plot_title', 'TBD'),
                'X_plot',
            )
        ], width=7),
        dbc.Col([
            make_plot(
                make_tooltip_title('Model output and expected output', 'y_plot_title', 'TBD'),
                'y_plot',
            )
        ], width=5),
    ], align='center',),
    html.Br(),
])

# callbacks

In [5]:
import plotly.express as px
from src.visualization import plot_map, intensity_map
from src.preprocessing import LabelCropp
from dash import Input, Output

@app.callback(
    Output('y_map_title', 'children'),
    Input('y_map_toggle', 'on'),
)
def update_y_map_title(toggle_position):
    return 'Expected map' if toggle_position else 'Predicted map'

"""
@app.callback(
    Output('left_map', 'figure'),
    Input("wavelength_slider", 'figure'),
    Input("map_toggle", "value"),
)
def update_left_map(slider, map_num):
    if map_num == 1:
        display_map = y_true
    else:
        display_map = y_pred
    left_map = intensity_map(
        display_map,
        dim=DIM,
        index_type=INDEX,
        start=slider[0],
        end=slider[1],
        calibration=calibration,
        suppress_outliers=True,
        colorscale=px.colors.sequential.haline,
    )
    left_map.update_layout(
        template='plotly_dark',
        plot_bgcolor= 'rgba(0, 0, 0, 0)',
        paper_bgcolor= 'rgba(0, 0, 0, 0)',
        yaxis=dict(scaleanchor='x'),
        margin=dict(l=0, r=0, b=0, t=0,),
    )

    return left_map
"""
"""
@app.callback(
    Output('difference_map', 'figure'),
    Input("wavelength_slider", "value"),
    Input("metric_dropdown", "value"),
)
def update_right_map(slider, metric_key):
    cropp = LabelCropp(label_from=slider[0], label_to=slider[1], labels=calibration)
    new_left = cropp.fit_transform(y_true)
    new_right = cropp.fit_transform(y_pred)
    values = METRICS[metric_key](new_left, new_right)
    right_map = plot_map(
        values,
        dim=DIM,
        index_type=INDEX,
        colorscale=px.colors.sequential.deep,
    )
    right_map.update_layout(
        template='plotly_dark',
        plot_bgcolor= 'rgba(0, 0, 0, 0)',
        paper_bgcolor= 'rgba(0, 0, 0, 0)',
        yaxis=dict(scaleanchor='x'),
        margin=dict(l=0, r=0, b=0, t=0,),
    )

    return right_map"""

'\n@app.callback(\n    Output(\'difference_map\', \'figure\'),\n    Input("wavelength_slider", "value"),\n    Input("metric_dropdown", "value"),\n)\ndef update_right_map(slider, metric_key):\n    cropp = LabelCropp(label_from=slider[0], label_to=slider[1], labels=calibration)\n    new_left = cropp.fit_transform(y_true)\n    new_right = cropp.fit_transform(y_pred)\n    values = METRICS[metric_key](new_left, new_right)\n    right_map = plot_map(\n        values,\n        dim=DIM,\n        index_type=INDEX,\n        colorscale=px.colors.sequential.deep,\n    )\n    right_map.update_layout(\n        template=\'plotly_dark\',\n        plot_bgcolor= \'rgba(0, 0, 0, 0)\',\n        paper_bgcolor= \'rgba(0, 0, 0, 0)\',\n        yaxis=dict(scaleanchor=\'x\'),\n        margin=dict(l=0, r=0, b=0, t=0,),\n    )\n\n    return right_map'

In [6]:
"""from src.visualization import id_from_snake_index, plot_spectra
from dash import callback_context

@app.callback(
    Output('spectra', 'figure'),
    Input('left_map', 'clickData'),
    Input('difference_map', 'clickData'),
)
def update_sample(left_click, right_click):
    ctx = callback_context.triggered[0]['prop_id'].split('.')[0]
    if left_click is None and right_click is None:
        x, y, id = 0, 0, 0
    elif ctx == 'difference_map':
        x, y = right_click['points'][0]['x'], right_click['points'][0]['y']
    elif ctx == 'left_map':
        x, y = left_click['points'][0]['x'], left_click['points'][0]['y']
    id = id_from_snake_index(x, y, DIM)

    fig = plot_spectra(
        np.vstack((y_true[id], y_pred[id])),
        calibration=calibration,
        labels=['left spectrum', 'right spectrum'],
        colormap=px.colors.qualitative.Set1,
        title=f"Sampled spectra for id={id} (x={x}, y={y})",
    )
    fig.update_layout(
        template='plotly_dark',
        plot_bgcolor= 'rgba(0, 0, 0, 0)',
        paper_bgcolor= 'rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, b=0, t=50,),
    )
    return fig"""

'from src.visualization import id_from_snake_index, plot_spectra\nfrom dash import callback_context\n\n@app.callback(\n    Output(\'spectra\', \'figure\'),\n    Input(\'left_map\', \'clickData\'),\n    Input(\'difference_map\', \'clickData\'),\n)\ndef update_sample(left_click, right_click):\n    ctx = callback_context.triggered[0][\'prop_id\'].split(\'.\')[0]\n    if left_click is None and right_click is None:\n        x, y, id = 0, 0, 0\n    elif ctx == \'difference_map\':\n        x, y = right_click[\'points\'][0][\'x\'], right_click[\'points\'][0][\'y\']\n    elif ctx == \'left_map\':\n        x, y = left_click[\'points\'][0][\'x\'], left_click[\'points\'][0][\'y\']\n    id = id_from_snake_index(x, y, DIM)\n\n    fig = plot_spectra(\n        np.vstack((y_true[id], y_pred[id])),\n        calibration=calibration,\n        labels=[\'left spectrum\', \'right spectrum\'],\n        colormap=px.colors.qualitative.Set1,\n        title=f"Sampled spectra for id={id} (x={x}, y={y})",\n    )\n 

# Run

In [7]:
if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
